In [1]:
config=3
# import module we'll need to import our custom module
from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
copyfile(src = "../input/fp2021/data.py", dst = "../working/data.py")
copyfile(src = "../input/fp2021/utils.py", dst = "../working/utils.py")
copyfile(src = "../input/fp2021/training.py", dst = "../working/training.py")
copyfile(src = "../input/fp2021/layers.py", dst = "../working/layers.py")
#copyfile(src = "../input/notes1/weights2.npy", dst = "../working/weights2.npy")
# import all our functions
#from data import load_data

'../working/layers.py'

In [2]:
!pip install pickle5
import pickle5 as pickle
import csv
import calendar
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import  MultipleLocator, FormatStrFormatter
from scipy.interpolate import splrep
from IPython.core.display import display_html
from keras.models import load_model
from utils import np_haversine, density_map, get_clusters, plot_embeddings
from data import load_data
#from training import start_new_session, process_features, create_model

# Display plots inline
%matplotlib inline

# Fix random seed for reproducibility
np.random.seed(42)

     |████████████████████████████████| 256 kB 5.2 MB/s            


In [3]:
def start_new_session():
    """
    Starts a new Tensorflow session.
    """
    
    # Make sure the session only uses the GPU memory that it actually needs
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    
    session = tf.compat.v1.Session(config=config, graph=tf.compat.v1.get_default_graph())
    tf.compat.v1.keras.backend.set_session(session)

In [4]:
clusters_cache = '/kaggle/input/fp2021/clusters.npy'
clusters=np.load(clusters_cache, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')

In [5]:
metadata_cache = '/kaggle/input/fp2021/metadata.pickle'
with open(metadata_cache, 'rb') as handle:
    metadata = pickle.load(handle)

In [6]:
file='/kaggle/input/fp2021/train0.npy'
train0=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/train1.npy'
train1=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/train2.npy'
train2=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/train3.npy'
train3=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/train4.npy'
train4=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/train5.npy'
train5=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/train6.npy'
train6=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
train=[train0,train1,train2,train3,train4,train5,train6]

In [7]:
file='/kaggle/input/fp2021/validation0.npy'
validation0=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/validation1.npy'
validation1=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/validation2.npy'
validation2=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/validation3.npy'
validation3=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/validation4.npy'
validation4=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/validation5.npy'
validation5=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
file='/kaggle/input/fp2021/validation6.npy'
validation6=np.load(file, mmap_mode=None, allow_pickle=False, fix_imports=True, encoding='ASCII')
validation=[validation0,validation1,validation2,validation3,validation4,validation5,validation6]

In [8]:
metadata_cache = '/kaggle/input/fp2021/metadata.pickle'
with open(metadata_cache, 'rb') as handle:
    metadata = pickle.load(handle)
tl_cache = '/kaggle/input/fp2021/train-labels.npy'
vl_cache = '/kaggle/input/fp2021/validation-labels.npy'
train_labels=np.load(tl_cache, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')
validation_labels=np.load(vl_cache, mmap_mode=None, allow_pickle=True, fix_imports=True, encoding='ASCII')

In [9]:
#import pickle
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import scale
from keras.models import Sequential
from tensorflow.keras.optimizers import SGD, Adam, Adagrad
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Reshape, Activation, Dropout
from keras.layers import *
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from utils import tf_haversine
from data import load_data
from utils import get_clusters
from keras.models import Model

from tensorflow.compat.v1.keras.layers import LSTM   # CuDNNLSTM
import sys
sys.path.append('..')  # add parent directory to Python path for layers.py access
from layers import Attention, SelfAttention

def create_model(metadata,clusters,config):
    """
    Creates all the layers for our neural network model.
    """
      
    # Arbitrary dimension for all embeddings
    embedding_dim = 10

    # Quarter hour of the day embedding
    embed_quarter_hour = Sequential()
    embed_quarter_hour.add(Embedding(metadata['n_quarter_hours'], embedding_dim, input_length=1))
    embed_quarter_hour.add(Reshape((embedding_dim,1)))

    # Day of the week embedding
    embed_day_of_week = Sequential()
    embed_day_of_week.add(Embedding(metadata['n_days_per_week'], embedding_dim, input_length=1))
    embed_day_of_week.add(Reshape((embedding_dim,1)))

    # Week of the year embedding
    embed_week_of_year = Sequential()
    embed_week_of_year.add(Embedding(metadata['n_weeks_per_year'], embedding_dim, input_length=1))
    embed_week_of_year.add(Reshape((embedding_dim,1)))

    # Client ID embedding
    embed_client_ids = Sequential()
    embed_client_ids.add(Embedding(metadata['n_client_ids'], embedding_dim, input_length=1))
    embed_client_ids.add(Reshape((embedding_dim,1)))

    # Taxi ID embedding
    embed_taxi_ids = Sequential()
    embed_taxi_ids.add(Embedding(metadata['n_taxi_ids'], embedding_dim, input_length=1))
    embed_taxi_ids.add(Reshape((embedding_dim,1)))

    # Taxi stand ID embedding
    embed_stand_ids = Sequential()
    embed_stand_ids.add(Embedding(metadata['n_stand_ids'], embedding_dim, input_length=1))
    embed_stand_ids.add(Reshape((embedding_dim,1)))
    
    # GPS coordinates (5 first lat/long and 5 latest lat/long, therefore 20 values)
    coords = Sequential()
    coords.add(Dense(1, input_dim=20))

    # Merge all the inputs into a single input layer
    mergedOut = Add()([embed_quarter_hour.output,
                embed_day_of_week.output,
                embed_week_of_year.output,
                embed_client_ids.output,
                embed_taxi_ids.output,
                embed_stand_ids.output,
                coords.output])
    
    #encoder_output, hidden_state, cell_state = LSTM(50, activation='tanh',input_shape=(None,None, 1),
    #                                                return_sequences=True,return_state=True)(mergedOut)
    #attention_input = [encoder_output, hidden_state]
    if config != 0:
        encoder_output, hidden_state, cell_state = LSTM(units=200,
                                                         return_sequences=True
                                                        ,return_state=True#,input_shape=(50,1)
                                                       )(mergedOut)
        attention_input = [encoder_output, hidden_state]
    else:
        encoder_output = LSTM(units=64)(mergedOut)

    # Optional Attention Mechanisms
    if config == 1:
        encoder_output, attention_weights = SelfAttention(size=64,
                                                      num_hops=10,
                                                      use_penalization=False)(encoder_output)
    elif config == 2:
        encoder_output, attention_weights = Attention(context='many-to-one',
                                                  alignment_type='global')(attention_input)
        encoder_output = Flatten()(encoder_output)
    elif config == 3:
        encoder_output, attention_weights = Attention(context='many-to-one',
                                                  alignment_type='local-m',
                                                  window_width=10,
                                                  score_function='general')(attention_input)
        encoder_output = Flatten()(encoder_output)
        
    
    #encoder_output=Dense(1)(encoder_output)
    # Determine cluster probabilities using softmax
    mergedOut=Dense(len(clusters))(encoder_output)
    mergedOut=Activation('softmax')(mergedOut)

    # Final activation layer: calculate the destination as the weighted mean of cluster coordinates
    cast_clusters = K.cast_to_floatx(clusters)
    def destination(probabilities):
        return tf.matmul(probabilities, cast_clusters)
    mergedOut=Activation(destination)(mergedOut)

    newModel = Model([embed_quarter_hour.input,
                embed_day_of_week.input,
                embed_week_of_year.input,
                embed_client_ids.input,
                embed_taxi_ids.input,
                embed_stand_ids.input,
                coords.input], mergedOut)
    #use lists if you want more than one input or output  
    
    # Compile the model
    optimizer = SGD(lr=0.1, momentum=0.9, clipvalue=1.)  # Use `clipvalue` to prevent exploding gradients

    newModel.compile(loss=tf_haversine, optimizer=optimizer)
    
    return newModel

In [10]:
model=create_model(metadata,clusters,config)


User settings:

   KMP_AFFINITY=granularity=fine,verbose,compact,1,0
   KMP_BLOCKTIME=0
   KMP_DUPLICATE_LIB_OK=True
   KMP_INIT_AT_FORK=FALSE
   KMP_SETTINGS=1
   KMP_WARNINGS=0

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=128
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=true
   KMP_ENABLE_TASK_THROTTLING=true
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_PLAIN_BARRIER='2,2'
   KMP_PLAIN_BARRIER_P

In [11]:
class CustCallback(tf.keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        global training_finished
        training_finished = True
    def on_epoch_end(self, epoch, logs=None):
        z=len(f)
        file_path="../working/Newweights"+str(z)
        weight=model.get_weights()
        np.save(file_path, weight)
        f.append(0)

In [12]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_train_end(self, logs=None):
        global training_finished
        training_finished = True
    def on_epoch_end(self, epoch, logs=None):
        z=len(e)
        if z>len(g):
            a=model.history.history['val_loss'][z-len(g)-1]
            b=model.history.history['loss'][z-len(g)-1]
            c=float(a)-float(b)
            if c<0:
                print("Sebelumnya Underfitting")
            else:
                print("Sebelumnya Overfitting")
        e.append(0)
            

In [13]:
e=[]
f=[]
g=[]
h=[]
for i in range(88):
    f.append(0)
    e.append(0)
    g.append(0)

In [14]:
file='../input/latl82fiturlocm/Newweights'+str(len(g)-1)+'.npy'
y=np.load(file,allow_pickle=True)
model.set_weights(y)

In [15]:
# Estimate the GPS clusters
print("Estimating clusters...")
#clusters = get_clusters(data.train_labels)
n_epochs=100
batch_size=200
save_prefix='mymodel'


Estimating clusters...


In [16]:
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.callbacks import EarlyStopping
 
callbacks = []
if save_prefix is not None:
        # Save the model's intermediary weights to disk after each epoch
    file_path="cache/%s-{epoch:03d}-{val_loss:.4f}.hdf5" % save_prefix
    checkpoint = ModelCheckpoint(file_path,monitor='val_loss',mode='auto',save_weights_only=True,verbose=0)
    #checkpoint = weight_save(model.get_weights(),b)
    callbacks.append(checkpoint)
    #callbacks.append(ModelCheckpoint(file_path, monitor='val_loss', mode='min', save_weights_only=True, verbose=1))
    #callbacks.append(ModelCheckpoint(file_path, monitor='val_loss', verbose=0, save_best_only=False, mode='auto'))
    #model.fit(X_train,y_train,batch_size=batch_size,nb_epoch=nb_epoch,callbacks=[weight_save_callback])
#g.append(0)
#epoch>1
#first epoch
#saver = weight_save(model.get_weights(),b)   
#es = EarlyStopping(monitor='val_loss', patience=0, verbose=1)
arc='AttenLSTMfitur91-b200-'+str(len(g))+'.log'
csv_logger = CSVLogger(arc, separator=',', append=False)
print("Creating model...")
start_new_session()
print("Train model...")
history=model.fit(train, train_labels,
        initial_epoch=len(g),epochs=100, batch_size=batch_size,
        validation_data=(validation, validation_labels)
        ,callbacks=[csv_logger,CustCallback(),MyCallback()])


Creating model...


2021-12-29 16:28:39.101746: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Train model...


2021-12-29 16:28:39.799582: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 89/100
6578/6578 [==============================] - 1478s 224ms/step - loss: 1.9230 - val_loss: 2.0060
Epoch 90/100


/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


6578/6578 [==============================] - 1474s 224ms/step - loss: 1.9220 - val_loss: 2.0097
Sebelumnya Overfitting
Epoch 91/100
6578/6578 [==============================] - 1468s 223ms/step - loss: 1.9245 - val_loss: 2.0134
Sebelumnya Overfitting
Epoch 92/100
6578/6578 [==============================] - 1520s 231ms/step - loss: 1.9227 - val_loss: 2.0125
Sebelumnya Overfitting
Epoch 93/100
6578/6578 [==============================] - 1463s 222ms/step - loss: 1.9204 - val_loss: 2.0215
Sebelumnya Overfitting
Epoch 94/100
6578/6578 [==============================] - 1466s 223ms/step - loss: 1.9198 - val_loss: 2.0138
Sebelumnya Overfitting
Epoch 95/100
6578/6578 [==============================] - 1465s 223ms/step - loss: 1.9208 - val_loss: 2.0239
Sebelumnya Overfitting
Epoch 96/100
6578/6578 [==============================] - 1464s 223ms/step - loss: 1.9219 - val_loss: 2.0047
Sebelumnya Overfitting
Epoch 97/100
6578/6578 [==============================] - 1476s 224ms/step - loss: 1.9191

In [17]:
t=0
while(t<1):
    t=t

KeyboardInterrupt: 

In [ ]:
#metadata_cache = '/kaggle/input/dataset91/train.pickle'
#with open(metadata_cache, 'rb') as handle:
#    train = pickle.load(handle)
#metadata_cache = '/kaggle/input/dataset91/validation.pickle'
#with open(metadata_cache, 'rb') as handle:
#    validation = pickle.load(handle)
import pickle5 as pickle
metadata_cache = '/kaggle/input/fp2021/competition-test.pickle'
with open(metadata_cache, 'rb') as handle:
    competition_test = pickle.load(handle)
train_labels=np.load('/kaggle/input/fp2021/train-labels.npy')
validation_labels=np.load('/kaggle/input/fp2021/validation-labels.npy')
competition_test_labels=np.load('/kaggle/input/fp2021/competition-test-labels.npy')
print("HASIL EVALUASI MODEL-INPUT 82 Full Fitur")
print(" ")
test_predictions = model.predict(process_features(competition_test))
print("Mean Haversine Test:{}".format(np_haversine(test_predictions, competition_test_labels).mean()))
validation_predictions = model.predict(validation)
print("Mean Haversine Validation:{}".format(np_haversine(validation_predictions, validation_labels).mean()))
train_predictions = model.predict(train)
print("Mean Haversine Train:{}".format(np_haversine(train_predictions, train_labels).mean()))
print(" ")
y_train=train_labels
y_train_pred=train_predictions
y_valid=validation_labels
y_valid_pred=validation_predictions
y_test=competition_test_labels
y_test_pred=test_predictions
from sklearn import metrics
print("MSE train:{}".format(metrics.mean_squared_error(y_train, y_train_pred)))
print("MSE validation:{}".format(metrics.mean_squared_error(y_valid, y_valid_pred)))
print("MSE test:{}".format(metrics.mean_squared_error(y_test, y_test_pred)))
print(" ")
print("MAE score train:{}".format(metrics.mean_absolute_error(y_train, y_train_pred)))
print("MAE score validation:{}".format(metrics.mean_absolute_error(y_valid, y_valid_pred)))
print("MAE score test:{}".format(metrics.mean_absolute_error(y_test, y_test_pred)))
print(" ")
print("R2 score train:{}".format(metrics.r2_score(y_train, y_train_pred)))
print("R2 score validation:{}".format(metrics.r2_score(y_valid, y_valid_pred)))
print("R2 score test:{}".format(metrics.r2_score(y_test, y_test_pred)))